In [1]:
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import cProfile as profile
import math
from tensorflow.keras.utils import plot_model
import random

In [2]:
#@title Class net structure

class NetStructure:
    def __init__(self, input_dim, output_dim):
        self.input_dim  = input_dim
        self.n_hidden   = 0
        self.hidden_dim = []
        self.output_dim = output_dim
        self.activation = []

    def add_hidden(self, hidden_dim, activation = 'linear'):
        self.n_hidden += 1
        self.hidden_dim.append(hidden_dim)
        self.activation.append(activation)

    def get_input_dim(self):
        return self.input_dim

    def get_output_dim(self):
        return self.output_dim

    def get_num_hidden(self):
        return self.n_hidden

    def get_hidden_dim(self, index):
        return self.hidden_dim[index]

    def get_activation(self, index):
        return self.activation[index]

    def print(self):
        print("----------------------")
        print("    Input dim:", self.input_dim)
        for i in range(self.n_hidden):
            print(" Hidden", i+1, "dim:", self.hidden_dim[i], "- activation:", self.activation[i])
        print("   Output dim:", self.output_dim)
        print("----------------------")

In [3]:
#@title Meta

class Meta:
    def __init__(self, net_structure):
        self.net_structure  = net_structure

        # I define the mdoel on the basis of the structure i handed it
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(net_structure.get_hidden_dim(0), activation=net_structure.get_activation(0), input_dim=net_structure.get_input_dim()))
        for i in range(1, net_structure.get_num_hidden()):
            self.model.add(tf.keras.layers.Dense(net_structure.get_hidden_dim(i), activation=net_structure.get_activation(i)))
        self.model.add(tf.keras.layers.Dense(net_structure.get_output_dim()))


        # save the number of model parameters
        self.num_parameters = self.model.count_params()

    def get_model(self):
        return self.model

    def set_num_iterations(self, num_iterations):
        self.num_iterations = num_iterations

    def set_population_size(self, population_size):
        self.population_size = population_size

    # def is_in_domain(self, x):
    #   if (x < self.domain[0] or x > self.domain[1]):
    #       return False
    #   return True

    def update_model_with_parameters(self, opt_par):
        nl = len(self.model.layers)
        wbindex = 0
        for p in range(0, nl):
          W = opt_par[wbindex:(wbindex + self.model.layers[p].input.shape[1] * self.model.layers[p].output.shape[1])]
          b = opt_par[(wbindex + self.model.layers[p].input.shape[1] * self.model.layers[p].output.shape[1]):(wbindex + self.model.layers[p].count_params())]
          self.model.layers[p].set_weights([W.reshape(self.model.layers[p].input.shape[1], self.model.layers[p].output.shape[1]), b])
          wbindex = (wbindex + self.model.layers[p].count_params())
    # the objective function used is MSE
    def objective_function(self, n_samples):
        X = np.random.uniform(low=0, high=1, size=(n_samples, 2))
        y_pred = self.model.predict(X, verbose = 0)
        Y = X[:,0]**2 + X[:,1]**2
        rest = np.mean((y_pred.flatten() - Y)**2)

        return rest

In [4]:
#@title Class GA

class GA(Meta):

    def __init__(self, net_structure):
        super().__init__(net_structure)
        self.x_rate = 0.60
        self.mutation_rate = 0.5

    def set_options(self, x_rate = 0.60, mutation_rate = 0.1):
        self.x_rate = x_rate
        self.mutation_rate = mutation_rate

    def set_max_x(self, max_x):
        self.max_x = max_x

    def natural_selection(self, population, costs):
      # transform the x_rate into a numerical value indicating how far up the population vector index I must go
        n = int(self.x_rate * self.population_size)
        costs = costs[:len(population)]
        indices = np.argsort(costs)
      # I sort the population according to the value of the cost function to identify which individuals are the best performers
        sorted_population = population[indices]
        selected_population = sorted_population[:n]
        return selected_population

    #mating functions: Roulette Wheel weighting
    def roulette_wheel_weighting(self, population, costs):
        probability = []
        costs =  np.sort(costs)
        ordinated_costs = costs[:len(population)]
        cost_n = costs[-1]
        for i in range(len(ordinated_costs)):
    # I normalize the values of the cost function so that the sum is one
            probability.append((ordinated_costs[i] - cost_n) / (sum(ordinated_costs) - (cost_n * len(ordinated_costs))))
        rand = np.random.uniform(probability[-1], 1)
    # find the chromosome that corresponds to the interval in which rand falls
        for q in range(len(probability)):
            if rand > probability[q]:
                chosen_chromosome =  population[q]
                return chosen_chromosome


    def mating(self, population, costs):
        father = self.roulette_wheel_weighting(population, costs)
        mother = self.roulette_wheel_weighting(population, costs)
        beta = np.random.uniform(low = 0, high = 10, size = (self.num_parameters))
        #crossover: blending method
        offspring_1 = father - np.multiply(beta, mother - father)
        offspring_2 = mother + np.multiply(beta, mother - father)
      # adding new generations to the population
        population = np.vstack((population, offspring_1))
        population = np.vstack((population, offspring_2))
        return population

    def mutation(self, population, best_index):
    # number of mutati
        mutation_number = int(self.mutation_rate * self.num_parameters * len(population))
    # repeat mutation_number times: I take any chromosome and modify any gene of it
        for t in range(mutation_number):
            chromosome_choice = np.random.randint(0, len(population)-1)
            if t == chromosome_choice and best_index != t:
                for p in range(len(population)):
                    gene_choice = np.random.randint(0, self.num_parameters-1)
                    if p == gene_choice:
                        population[t][gene_choice] = np.random.uniform(low = -self.max_x, high = self.max_x)
        return population


    def optimize(self):
        population = np.random.uniform(low = -self.max_x, high = self.max_x, size = (self.population_size, self.num_parameters ))
        best_positions = np.copy(population)
        best_scores = np.array([self.num_parameters] * self.population_size)
        global_best_position = np.copy(population[0])
        global_best_score = 1e10
        nl = len(self.model.layers)
        len_population = len(population)
        costs = np.zeros(len(population))

        for iteration in range(self.num_iterations):
            tic_global = time.perf_counter()

            for i in range(len_population):
                genome = population[i,]
                best_index = 0

                wbindex = 0

                for p in range(0, nl):
                  W = genome[wbindex:(wbindex + self.model.layers[p].input.shape[1] * self.model.layers[p].output.shape[1])]
                  b = genome[(wbindex + self.model.layers[p].input.shape[1] * self.model.layers[p].output.shape[1]):(wbindex + self.model.layers[p].count_params())]
                  self.model.layers[p].set_weights([W.reshape(self.model.layers[p].input.shape[1], self.model.layers[p].output.shape[1]), b])
                  wbindex = (wbindex + self.model.layers[p].count_params())


                fitness = self.objective_function(n_samples=1000)
                costs[i] = self.objective_function(n_samples=1000)



                # print(fitness)

                if  fitness < best_scores[i]:
                    best_scores[i] = fitness
                    best_positions[i] = np.copy(population[i])

                if  fitness < global_best_score:
                    global_best_score = fitness
                    best_index = i
                    global_best_position = np.copy(population[i])


            population = self.natural_selection(population, costs)
            population = self.mating(population, costs)
            population = self.mutation(population, best_index)

            toc_global = time.perf_counter()
            len_population = len(population)

            # check that the adjourned positions are not outside the domain
            population  = np.minimum(population,  self.max_x)
            population  = np.maximum(population, -self.max_x)


            print("Iteration #%d - Objective function value: %5.2f - time: %0.3f " % (iteration, global_best_score, toc_global - tic_global))
            if (global_best_score == 0):
                break


        return global_best_position

    def predict(self, x = None):
        return self.model.predict(x)

In [5]:
#@title Code for traing and test with GA

net = NetStructure(input_dim=2, output_dim=1)
net.add_hidden(hidden_dim=8)
net.add_hidden(hidden_dim=8)


met = GA(net)
met.set_num_iterations(5)
met.set_population_size(200)
met.set_max_x(1)


sample_size = 100
domain = [-1, 1]

# TRAIN
optimized_params = met.optimize()
met.update_model_with_parameters(optimized_params)


X_test = np.random.uniform(low=1, high=2, size=(sample_size, 2))

# TEST
y_pred_test = met.predict(X_test)
mse_test = np.mean((y_pred_test.flatten() - y_pred_test)**2)
print(f"Mean Squared Error on Test Set: {mse_test}")



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Iteration #0 - Objective function value:  0.05 - time: 62.891 
Iteration #1 - Objective function value:  0.05 - time: 34.880 
Iteration #2 - Objective function value:  0.05 - time: 42.757 
Iteration #3 - Objective function value:  0.04 - time: 40.575 
Iteration #4 - Objective function value:  0.04 - time: 41.335 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Mean Squared Error on Test Set: 0.24322812259197235


In [6]:
#@title Class WOA

class WOA(Meta):

    def __init__(self, net_structure):
        super().__init__(net_structure)
        self.b  = 1 # spiral_shape_param

    def set_options(self, spiral_shape_param = 1):
        self.b = spiral_shape_param

    def set_max_x(self, max_x):
        self.max_x = max_x
    # I update the position using the equations determined from the case corresponding to the values of p andA
    def update_position(self, A, global_best_position, whales, C, l, p, i ):
        if p < 0.5:
            if abs(A) < 1:
                D = abs(C * global_best_position - whales[i])
                new_position = global_best_position - A * D
            else:
                X_rand = np.random.uniform(-self.max_x, self.max_x)
                D = abs(C * X_rand - whales[i])
                new_position = C * X_rand - whales[i]
        else:
            D = abs(global_best_position - whales[i])
            new_position = global_best_position + D * math.exp(self.b * l) * math.cos(2 * math.pi * l)
        return new_position


    def optimize(self):
        whales = np.random.uniform(low=-self.max_x, high=self.max_x, size=(self.population_size, self.num_parameters))
        best_positions = np.copy(whales)
        best_scores = np.array([self.num_parameters] * self.population_size)
        global_best_position = np.copy(whales[0])
        global_best_score = 1e10
        nl = len(self.model.layers)

        for iteration in range(self.num_iterations):
            tic_global = time.perf_counter()
            counter = 0

            for i in range(self.population_size):
                whale = whales[i,]
                wbindex = 0
                # WOA paramters
                r = np.random.randn()
                a = 2 - iteration * (2 / self.num_iterations)
                A = 2 * a * r - a
                C = 2 * r
                p = np.random.rand()
                l = np.random.uniform(-1,1)


                for p in range(0, nl):
                    W = whale[wbindex:(wbindex + self.model.layers[p].input.shape[1] * self.model.layers[p].output.shape[1])]
                    b = whale[(wbindex + self.model.layers[p].input.shape[1] * self.model.layers[p].output.shape[1]):(wbindex + self.model.layers[p].count_params())]
                    self.model.layers[p].set_weights([W.reshape(self.model.layers[p].input.shape[1], self.model.layers[p].output.shape[1]), b])
                    wbindex = (wbindex + self.model.layers[p].count_params())

                fitness = self.objective_function(n_samples = 1000)

                if  fitness < best_scores[i]:
                    best_scores[i] = fitness
                    best_positions[i] = np.copy(whales[i])


                if  fitness < global_best_score:
                    global_best_score = fitness
                    global_best_position = np.copy(whales[i])

                whales[i] = self.update_position(A, global_best_position, whales, C, l, p, i)

            # check that the adjourned positions are not outside the domain
            whales  = np.minimum(whales,  self.max_x)
            whales  = np.maximum(whales, -self.max_x)

            toc_global = time.perf_counter()
            print("Iteration #%d - Objective function value: %5.2f - time: %0.3f " % (iteration, global_best_score, toc_global - tic_global))

            if (global_best_score == 0):
                break

        return global_best_position

    def predict(self, x = None):
        return self.model.predict(x)

In [7]:
#@title Code for traing and test with WOA

net = NetStructure(input_dim=2, output_dim=1)
net.add_hidden(hidden_dim=5)


met = WOA(net)
met.set_num_iterations(30)
met.set_population_size(10)
met.set_max_x(2)

sample_size = 1000
domain = [-1, 1]

# TRAIN
optimized_params = met.optimize()
met.update_model_with_parameters(optimized_params)

X_test = np.random.uniform(low=1, high=2, size=(sample_size, 2))

# TEST
y_pred_test = met.predict(X_test)
mse_test = np.mean((y_pred_test.flatten() - y_pred_test)**2)
print(f"Mean Squared Error on Test Set: {mse_test}")


Iteration #0 - Objective function value:  1.65 - time: 1.895 
Iteration #1 - Objective function value:  1.65 - time: 1.939 
Iteration #2 - Objective function value:  0.88 - time: 1.178 
Iteration #3 - Objective function value:  0.54 - time: 1.193 
Iteration #4 - Objective function value:  0.14 - time: 1.271 
Iteration #5 - Objective function value:  0.14 - time: 1.346 
Iteration #6 - Objective function value:  0.02 - time: 1.314 
Iteration #7 - Objective function value:  0.02 - time: 1.217 
Iteration #8 - Objective function value:  0.02 - time: 1.234 
Iteration #9 - Objective function value:  0.02 - time: 1.503 
Iteration #10 - Objective function value:  0.02 - time: 1.934 
Iteration #11 - Objective function value:  0.02 - time: 1.380 
Iteration #12 - Objective function value:  0.02 - time: 1.354 
Iteration #13 - Objective function value:  0.02 - time: 1.377 
Iteration #14 - Objective function value:  0.02 - time: 1.273 
Iteration #15 - Objective function value:  0.02 - time: 1.308 
It

In [8]:
#@title Class PSO

class PSO(Meta):
    def __init__(self, net_structure):
        super().__init__(net_structure)
        self.w  = 0.3 # inertia_param
        self.c1 = 1.5 # cognitive_param
        self.c2 = 1.5 # social_param

    def set_options(self, inertia_param = 0.3,
                    cognitive_param = 1.5,
                    social_param = 1.5):
        self.w  = inertia_param
        self.c1 = cognitive_param
        self.c2 = social_param

    def set_max_v(self, max_v):
        self.max_v = max_v

    def set_max_x(self, max_x):
        self.max_x = max_x
    # updating speeds by calculating the three basic components: inertia, cogninitve_component and social_component
    def update_velocity(self, position, velocity, best_position, global_best_position):
        inertia = self.w * velocity
        cognitive_component = self.c1 *2* np.random.rand(1, len(position)) * (best_position - position)
        social_component = self.c2 *2* np.random.rand(1, len(position)) * (global_best_position - position)
        new_velocity = inertia + cognitive_component + social_component
        return new_velocity

    def optimize(self):
        particles  = np.random.uniform(low=-self.max_x, high=self.max_x, size=(self.population_size, self.num_parameters))
        velocities = np.random.uniform(low=-self.max_v, high=self.max_v, size=(self.population_size, self.num_parameters))
        best_positions = np.copy(particles)
        best_scores = np.array([self.num_parameters] * self.population_size)
        global_best_position = None
        global_best_score = 1e10
        nl = len(self.model.layers)

        for iteration in range(self.num_iterations):
            tic_global = time.perf_counter()

            for i in range(self.population_size):
                particle = particles[i,]

                wbindex = 0

                for p in range(0, nl):
                  W = particle[wbindex:(wbindex + self.model.layers[p].input.shape[1] * self.model.layers[p].output.shape[1])]
                  b = particle[(wbindex + self.model.layers[p].input.shape[1] * self.model.layers[p].output.shape[1]):(wbindex + self.model.layers[p].count_params())]
                  self.model.layers[p].set_weights([W.reshape(self.model.layers[p].input.shape[1], self.model.layers[p].output.shape[1]), b])
                  wbindex = (wbindex + self.model.layers[p].count_params())

                fitness = self.objective_function(n_samples = 1000)

                if  fitness < best_scores[i]:
                    best_scores[i] = fitness
                    best_positions[i] = np.copy(particles[i])

                if  fitness < global_best_score:
                    global_best_score = fitness
                    global_best_position = np.copy(particles[i])

                velocities[i] = self.update_velocity(particles[i], velocities[i], best_positions[i], global_best_position)
                particles[i] += velocities[i]
            # check that the adjourned positions are not outside the domain and velocities are not over the limit
            velocities = np.minimum(velocities,  self.max_v)
            velocities = np.maximum(velocities, -self.max_v)
            particles  = np.minimum(particles,  self.max_x)
            particles  = np.maximum(particles, -self.max_x)

            toc_global = time.perf_counter()
            print("Iteration #%d - Objective function value: %5.2f - time: %0.3f" % (iteration, global_best_score, toc_global - tic_global))

            if (global_best_score == 0):
                break
        return global_best_position

    def predict(self, x = None):
        return self.model.predict(x)

In [9]:
#@title Code for traing and test with PSO

net = NetStructure(input_dim=2, output_dim=1)
net.add_hidden(hidden_dim=5)

met = PSO(net)
met.set_num_iterations(20)
met.set_population_size(50)
met.set_max_v(0.3)
met.set_max_x(1.5)

sample_size = 1000
domain = [-1, 1]

# TRAIN
optimized_params = met.optimize()
met.update_model_with_parameters(optimized_params)

X_test = np.random.uniform(low=1, high=2, size=(sample_size, 2))

# TEST
y_pred_test = met.predict(X_test)
mse_test = np.mean((y_pred_test.flatten() - y_pred_test)**2)
print(f"Mean Squared Error on Test Set: {mse_test}")




Iteration #0 - Objective function value:  0.14 - time: 7.260
Iteration #1 - Objective function value:  0.13 - time: 7.589
Iteration #2 - Objective function value:  0.13 - time: 9.065
Iteration #3 - Objective function value:  0.13 - time: 7.402
Iteration #4 - Objective function value:  0.07 - time: 7.733
Iteration #5 - Objective function value:  0.04 - time: 6.600
Iteration #6 - Objective function value:  0.04 - time: 8.734
Iteration #7 - Objective function value:  0.04 - time: 8.641
Iteration #8 - Objective function value:  0.04 - time: 7.324
Iteration #9 - Objective function value:  0.04 - time: 7.535
Iteration #10 - Objective function value:  0.04 - time: 6.721
Iteration #11 - Objective function value:  0.04 - time: 8.287
Iteration #12 - Objective function value:  0.04 - time: 8.858
Iteration #13 - Objective function value:  0.04 - time: 7.276
Iteration #14 - Objective function value:  0.04 - time: 7.693
Iteration #15 - Objective function value:  0.04 - time: 6.411
Iteration #16 - Ob